#Regressions

## Part 3.a
Write down a fixed effects regression equation that allows you to derive a difference-
in-difference estimate of the effect of the event on prices. Think of controls to
add, why is this relevant? Explain why you need a second city for this.

In [7]:
#Libraries that we are goint to use:
import pandas as pd
#!pip install statsmodels

import statsmodels.api as sm
import statsmodels.formula.api as smf


In [ ]:
df= pd.read_csv("///")

First lets run the most basic regression:

In [ ]:



# Create the interaction term (DiD variable)
df["MCW*treated"] = df["MWC"] * df["treated"]

#Fist we will run a regression without CONTROLS.

model = smf.ols(
    "price ~ post + treated + post_treated + C(city) + C(time)", 
    data=df
).fit()

# Print summary results
print(model.summary())


## 3.b


How would you use text features from the description as controls? Think about
the text in the descriptions you scraped. How would this help? Why would terms
like "Barcelona" not help?

Use new controls as: 
"piscina" , "wifi","spa", "lujoso"  "cerca del centro" ,"cerca del metro"
More if you wand

In [ ]:
#HERE the data frame is not the same that we had because now we will have the columns of the frequency of the data
model2=smf.ols(
    "price ~ post + treated + post_treated + C(city) + C(time) + X + XX + XXX+ XXX",
    data=df
).fit()

- **Why bacelona not help?**  : Almost Every hotel in the barcelona  mention "Barcelona" and making it non-informative. the word Barcelona doesn’t add variation to the model.

## 3.c


Now suppose we want to decompose the treatment effect by hotel quality. Can
you use the text description here? How would you use them to study heterogenous
treatment effects? Write down a regression equation.

WE can add another column that represents the quality. Quality =2 represents Luxury .Quality=1 represens Mid-range,  Quality = 0 represent Budget hotels

Luxury hotels (mentioning "5-star", "spa", "rooftop", etc.).
Mid-range hotels ("comfortable", "good location", "breakfast included").
Budget hotels ("affordable", "hostel", "shared room").

In [ ]:
#need to generate another model where the control columns are some